In [1]:
import pandas as pd
import re

In [2]:
df_data = pd.read_csv("Asset Challenge/data.csv",encoding='latin1')
df_alay = pd.read_csv("Asset Challenge/new_kamusalay.csv",encoding='latin1',header=None)
df_kasar = pd.read_csv("Asset Challenge/abusive.csv",encoding='latin1')

In [3]:
#Membaca semua dataset yang ada
df_alay=df_alay.rename(columns={0:'Original',1:'Baku'})

df_kasar=pd.read_csv("Asset Challenge/abusive.csv",encoding='latin1')

df_kasar['Kata_Sensor']="53544e534f52" #Inisiasi kata ganti untuk kata-kata yang kasar dengan kata "disensor"

alay_dict_map = dict(zip(df_alay['Original'], df_alay['Baku']))
kasar_dict_map = dict(zip(df_kasar['ABUSIVE'],df_kasar['Kata_Sensor']))

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nlp_id.lemmatizer import Lemmatizer 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])


def sensor_kata_kasar(text):
    return ' '.join([kasar_dict_map[word] if word in kasar_dict_map else word for word in text.split(' ')])


def preprocess(TextYangInginDiPreProcess):
    #Tahap Pertama Adalah Membuat semua huruf menjadi huruf kecil atau lower
    text = TextYangInginDiPreProcess.lower()

    #Tahap Kedua adalah menghilangkan non alpha numeric character pada text
    text = re.sub('[^0-9a-zA-Z]+',' ',text)

    #Tahap Ketiga adalah menghilangkan char tidak penting
    text=re.sub('\n',' ',text) #Menghilangkan new line pada data
    text=re.sub('rt',' ',text) #Menghilangkan kata-kata retweet 
    text=re.sub('user',' ',text) #Menghilangkan kata-kata user
    text=re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) #Menghilangkan  URL
    text=re.sub(' +',' ',text) #Menghilangkan ekstra spasi

    #Tahap keempat adalah membuat map terhadap kata-kata "alay" dan mengubah nya menjadi kata yang baku
    text=normalize_alay(text)

    #Tahap kelima adalah mensensor kata kasar dengan kata "disensor"
    text=sensor_kata_kasar(text)
    return text

def menghilangkan_kata_umum_dengan_stopword(text):
    # Mengambil daftar stopwords bahasa Indonesia dari NLTK
    stop_words = set(stopwords.words('indonesian'))
    
    # Memisahkan teks menjadi kata-kata
    words = text.split()
    
    # Menghapus stopwords dari teks
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    # Menggabungkan kata-kata yang tersisa menjadi teks tanpa stopwords
    filtered_text = ' '.join(filtered_words)
    
    return filtered_text

def mengurangi_variasi_kata_dengan_lematisasi(teks):
    lemmatizer = Lemmatizer() 
    teks = lemmatizer.lemmatize(teks) 
    return teks

def preprocess_semua_kata(teks):
    teks = preprocess(teks)
    teks = menghilangkan_kata_umum_dengan_stopword(teks)
    teks = mengurangi_variasi_kata_dengan_lematisasi(teks)
    return teks

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/raihanekaputra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df_data_1 = df_data.iloc[:2000]
df_data_1

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,USER Ga sekalian lu jadiin gua anak nih biar m...,0,1,0,0,0,0,0,0,0,0,0,0
1996,RT USER: Heran sama orang yang maksa maksa min...,0,0,0,0,0,0,0,0,0,0,0,0
1997,mmg kalo gak pernah turun langsung ke lapangan...,1,1,1,0,0,1,0,0,0,1,0,0
1998,USER USER USER Lebih sadis gerombolan elo saat...,1,1,0,1,0,0,0,0,1,0,1,0


In [6]:
# Menguji Fungsi Untuk Mempreprocess Kata-Kata
kata_kata = "Ini Adalah Anak Anjing"
kata_setelah_di_preprocess = preprocess_semua_kata(kata_kata)
print(kata_setelah_di_preprocess)

anak 53544e534f52


In [7]:
df_data = df_data[:2000]
df_data['Tweet']=df_data['Tweet'].apply(preprocess_semua_kata)

df_data_1_tweet_hs = df_data[['Tweet','HS']]

df_data_2_tweet_jenis_hs = df_data[['Tweet','HS_Individual','HS_Group',
                                    'HS_Religion','HS_Race','HS_Physical','HS_Gender','HS_Other']]

df_data_3_tweet_kekuatan_hs=df_data[['Tweet','HS_Weak','HS_Moderate','HS_Strong']]


In [8]:
df_data_1_tweet_hs #Dataset yang berfungsi untuk membedakan antara sebuah tweet yang terkategori sebagai hate speech

,Tweet,HS
0,cowok usaha lacak perhati gue lantas remeh per...,1
1,telat tau 53544e534f52 53544e534f52 gue gaul c...,0
2,41 kadang pikir percaya tuhan jatuh kali kali ...,0
3,ku tau mata 53544e534f52,0
4,kaum 53544e534f52 53544e534f52 dongok 53544e53...,1
...,...,...
1995,jadi gue anak nih biar 53544e534f52,0
1996,heran orang paksa paksa follow beda ya follow ...,0
1997,turun langsung lapang sungguh pantas jongos ci...,1
1998,sadis gerombol susilo bambang yudhoyono kuasa ...,1


In [9]:
df_data_2_tweet_jenis_hs

,Tweet,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,cowok usaha lacak perhati gue lantas remeh per...,1,0,0,0,0,0,1
1,telat tau 53544e534f52 53544e534f52 gue gaul c...,0,0,0,0,0,0,0
2,41 kadang pikir percaya tuhan jatuh kali kali ...,0,0,0,0,0,0,0
3,ku tau mata 53544e534f52,0,0,0,0,0,0,0
4,kaum 53544e534f52 53544e534f52 dongok 53544e53...,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
1995,jadi gue anak nih biar 53544e534f52,0,0,0,0,0,0,0
1996,heran orang paksa paksa follow beda ya follow ...,0,0,0,0,0,0,0
1997,turun langsung lapang sungguh pantas jongos ci...,1,0,0,1,0,0,0
1998,sadis gerombol susilo bambang yudhoyono kuasa ...,0,1,0,0,0,0,1


In [10]:
df_data_3_tweet_kekuatan_hs

,Tweet,HS_Weak,HS_Moderate,HS_Strong
0,cowok usaha lacak perhati gue lantas remeh per...,1,0,0
1,telat tau 53544e534f52 53544e534f52 gue gaul c...,0,0,0
2,41 kadang pikir percaya tuhan jatuh kali kali ...,0,0,0
3,ku tau mata 53544e534f52,0,0,0
4,kaum 53544e534f52 53544e534f52 dongok 53544e53...,0,1,0
...,...,...,...,...
1995,jadi gue anak nih biar 53544e534f52,0,0,0
1996,heran orang paksa paksa follow beda ya follow ...,0,0,0
1997,turun langsung lapang sungguh pantas jongos ci...,1,0,0
1998,sadis gerombol susilo bambang yudhoyono kuasa ...,0,1,0


In [11]:
#preprocess_semua_kata((df_data['Tweet']))

In [12]:
df_data_1_tweet_hs.to_csv('df_data_1_tweet_hs.csv',index=False)
df_data_2_tweet_jenis_hs.to_csv("df_data_2_tweet_jenis_hs.csv",index=False)
df_data_3_tweet_kekuatan_hs.to_csv("df_data_3_tweet_kekuatan_hs.csv",index=False)

In [13]:
tweet_hs_df = pd.read_csv("df_data_1_tweet_hs.csv",encoding='latin1')
tweet_jenis_hs = pd.read_csv("df_data_2_tweet_jenis_hs.csv",encoding='latin1')
tweet_kekuatan_hs = pd.read_csv("df_data_3_tweet_kekuatan_hs.csv",encoding='latin1')

In [14]:
tweet_hs_df

,Tweet,HS
0,cowok usaha lacak perhati gue lantas remeh per...,1
1,telat tau 53544e534f52 53544e534f52 gue gaul c...,0
2,41 kadang pikir percaya tuhan jatuh kali kali ...,0
3,ku tau mata 53544e534f52,0
4,kaum 53544e534f52 53544e534f52 dongok 53544e53...,1
...,...,...
1995,jadi gue anak nih biar 53544e534f52,0
1996,heran orang paksa paksa follow beda ya follow ...,0
1997,turun langsung lapang sungguh pantas jongos ci...,1
1998,sadis gerombol susilo bambang yudhoyono kuasa ...,1


In [15]:
tweet_jenis_hs

,Tweet,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,cowok usaha lacak perhati gue lantas remeh per...,1,0,0,0,0,0,1
1,telat tau 53544e534f52 53544e534f52 gue gaul c...,0,0,0,0,0,0,0
2,41 kadang pikir percaya tuhan jatuh kali kali ...,0,0,0,0,0,0,0
3,ku tau mata 53544e534f52,0,0,0,0,0,0,0
4,kaum 53544e534f52 53544e534f52 dongok 53544e53...,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
1995,jadi gue anak nih biar 53544e534f52,0,0,0,0,0,0,0
1996,heran orang paksa paksa follow beda ya follow ...,0,0,0,0,0,0,0
1997,turun langsung lapang sungguh pantas jongos ci...,1,0,0,1,0,0,0
1998,sadis gerombol susilo bambang yudhoyono kuasa ...,0,1,0,0,0,0,1


In [16]:
tweet_kekuatan_hs

,Tweet,HS_Weak,HS_Moderate,HS_Strong
0,cowok usaha lacak perhati gue lantas remeh per...,1,0,0
1,telat tau 53544e534f52 53544e534f52 gue gaul c...,0,0,0
2,41 kadang pikir percaya tuhan jatuh kali kali ...,0,0,0
3,ku tau mata 53544e534f52,0,0,0
4,kaum 53544e534f52 53544e534f52 dongok 53544e53...,0,1,0
...,...,...,...,...
1995,jadi gue anak nih biar 53544e534f52,0,0,0
1996,heran orang paksa paksa follow beda ya follow ...,0,0,0
1997,turun langsung lapang sungguh pantas jongos ci...,1,0,0
1998,sadis gerombol susilo bambang yudhoyono kuasa ...,0,1,0


## Sistem akan dibuat untuk menganalisis sentimen dengan tahap sebagai berikut
## 1. Analisis tweet apakah suatu tweet termasuk HS atau tidak
## 2. Jika termasuk HS, termasuk jenis HS apakah data itu
## 3. Jika termasuk HS, bagaimana kekuatan dari hate speech tersebut

### Memproses Data Tweet dan HS

In [30]:
jumlah_1 = tweet_hs_df['HS'].value_counts()[0]
jumlah_2 = tweet_hs_df['HS'].value_counts()[1]

tweet_hs_df_0 = tweet_hs_df[tweet_hs_df['HS'] == 0].sample(851,replace=True)
tweet_hs_df_1 = tweet_hs_df[tweet_hs_df['HS'] == 1].sample(851,replace=True)

tweet_hs_df = pd.concat([tweet_hs_df_0,tweet_hs_df_1])

print("Jumlah HS yang bernilai 0 adalah : "+str(jumlah_1))
print("Jumlah HS yang bernilai 1 adalah : "+str(jumlah_2))
tweet_hs_df

Jumlah HS yang bernilai 0 adalah : 851
Jumlah HS yang bernilai 1 adalah : 851


,Tweet,HS
1289,53544e534f52 nih haha kena nor haha karipap ka...,0
1725,xf0 x9f x8c xb9tomohon pusat ajar agama kriste...,0
792,53544e534f52 gue cowok cowok pakai baju merah ...,0
783,waspada jawa indikasi mobilisasi massa jawa ta...,0
916,otak 53544e534f52,0
...,...,...
50,sih otak tempurung laden lawan 53544e534f52 ma...,1
1050,tau junjung 53544e534f52 licik ya hamba tukang...,1
1410,gojek 2019 ganti presiden ayo viralkan cc,1
865,pagu ajak gelut 53544e534f52,1


In [23]:
print("Jumlah HS yang bernilai 0 adalah : "+str(jumlah_1))
print("Jumlah HS yang bernilai 1 adalah : "+str(jumlah_2))

Jumlah HS yang bernilai 0 adalah : 851
Jumlah HS yang bernilai 1 adalah : 851


### Memproses Data Tweet dan Jenis HS

In [29]:
jumlah_hs_individual_1 = tweet_jenis_hs['HS_Individual'].value_counts()[1]
jumlah_hs_group_1 = tweet_jenis_hs['HS_Group'].value_counts()[1]
jumlah_hs_religion_1 = tweet_jenis_hs['HS_Religion'].value_counts()[1]
jumlah_hs_race_1 = tweet_jenis_hs['HS_Race'].value_counts()[1]
jumlah_hs_physical_1 = tweet_jenis_hs['HS_Physical'].value_counts()[1]
jumlah_hs_gender_1 = tweet_jenis_hs['HS_Gender'].value_counts()[1]
jumlah_hs_others_1 = tweet_jenis_hs['HS_Other'].value_counts()[1]

jumlah_hs_individual_0 = tweet_jenis_hs['HS_Individual'].value_counts()[0]
jumlah_hs_group_0 = tweet_jenis_hs['HS_Group'].value_counts()[0]
jumlah_hs_religion_0 = tweet_jenis_hs['HS_Religion'].value_counts()[0]
jumlah_hs_race_0 = tweet_jenis_hs['HS_Race'].value_counts()[0]
jumlah_hs_physical_0 = tweet_jenis_hs['HS_Physical'].value_counts()[0]
jumlah_hs_gender_0 = tweet_jenis_hs['HS_Gender'].value_counts()[0]
jumlah_hs_others_0 = tweet_jenis_hs['HS_Other'].value_counts()[0]

print("Jumlah HS individual pada data adalah 1 : "+str(jumlah_hs_individual_1))
print("Jumlah HS group pada data adalah 1 : "+str(jumlah_hs_group_1))
print("Jumlah HS religion pada data adalah 1 : "+str(jumlah_hs_religion_1))
print("Jumlah HS physical pada data adalah 1 : "+str(jumlah_hs_physical_1))
print("Jumlah HS gender pada data adalah 1 : "+str(jumlah_hs_gender_1))
print("Jumlah HS others pada data adalah 1 : "+str(jumlah_hs_others_1))
print("-"*50)
print("Jumlah HS individual pada data adalah 0 : "+str(jumlah_hs_individual_0))
print("Jumlah HS group pada data adalah 0 : "+str(jumlah_hs_group_0))
print("Jumlah HS religion pada data adalah 0 : "+str(jumlah_hs_religion_0))
print("Jumlah HS physical pada data adalah 0 : "+str(jumlah_hs_physical_0))
print("Jumlah HS gender pada data adalah 0 : "+str(jumlah_hs_gender_0))
print("Jumlah HS others pada data adalah 0 : "+str(jumlah_hs_others_0))

Jumlah HS individual pada data adalah 1 : 523
Jumlah HS group pada data adalah 1 : 328
Jumlah HS religion pada data adalah 1 : 128
Jumlah HS physical pada data adalah 1 : 53
Jumlah HS gender pada data adalah 1 : 61
Jumlah HS others pada data adalah 1 : 551
--------------------------------------------------
Jumlah HS individual pada data adalah 0 : 1477
Jumlah HS group pada data adalah 0 : 1672
Jumlah HS religion pada data adalah 0 : 1872
Jumlah HS physical pada data adalah 0 : 1947
Jumlah HS gender pada data adalah 0 : 1939
Jumlah HS others pada data adalah 0 : 1449


In [36]:
tweet_jenis_hs = pd.concat([
    tweet_jenis_hs[tweet_jenis_hs['HS_Group'] == 0].sample(53, replace=True),
    tweet_jenis_hs[tweet_jenis_hs['HS_Religion'] == 0].sample(53, replace=True),
    tweet_jenis_hs[tweet_jenis_hs['HS_Race'] == 0].sample(53, replace=True),
    tweet_jenis_hs[tweet_jenis_hs['HS_Physical'] == 0].sample(53, replace=True),
    tweet_jenis_hs[tweet_jenis_hs['HS_Gender'] == 0].sample(53, replace=True),
    tweet_jenis_hs[tweet_jenis_hs['HS_Other'] == 0].sample(53, replace=True),
    tweet_jenis_hs[tweet_jenis_hs['HS_Group'] == 1].sample(53, replace=True),
    tweet_jenis_hs[tweet_jenis_hs['HS_Religion'] == 1].sample(53, replace=True),
    tweet_jenis_hs[tweet_jenis_hs['HS_Race'] == 1].sample(53, replace=True),
    tweet_jenis_hs[tweet_jenis_hs['HS_Physical'] == 1].sample(53, replace=True),
    tweet_jenis_hs[tweet_jenis_hs['HS_Gender'] == 1].sample(53, replace=True),
    tweet_jenis_hs[tweet_jenis_hs['HS_Other'] == 1].sample(53, replace=True),
])

jumlah_hs_individual_1 = tweet_jenis_hs['HS_Individual'].value_counts()[1]
jumlah_hs_group_1 = tweet_jenis_hs['HS_Group'].value_counts()[1]
jumlah_hs_religion_1 = tweet_jenis_hs['HS_Religion'].value_counts()[1]
jumlah_hs_race_1 = tweet_jenis_hs['HS_Race'].value_counts()[1]
jumlah_hs_physical_1 = tweet_jenis_hs['HS_Physical'].value_counts()[1]
jumlah_hs_gender_1 = tweet_jenis_hs['HS_Gender'].value_counts()[1]
jumlah_hs_others_1 = tweet_jenis_hs['HS_Other'].value_counts()[1]

jumlah_hs_individual_0 = tweet_jenis_hs['HS_Individual'].value_counts()[0]
jumlah_hs_group_0 = tweet_jenis_hs['HS_Group'].value_counts()[0]
jumlah_hs_religion_0 = tweet_jenis_hs['HS_Religion'].value_counts()[0]
jumlah_hs_race_0 = tweet_jenis_hs['HS_Race'].value_counts()[0]
jumlah_hs_physical_0 = tweet_jenis_hs['HS_Physical'].value_counts()[0]
jumlah_hs_gender_0 = tweet_jenis_hs['HS_Gender'].value_counts()[0]
jumlah_hs_others_0 = tweet_jenis_hs['HS_Other'].value_counts()[0]

print("Jumlah HS individual pada data adalah 1 : "+str(jumlah_hs_individual_1))
print("Jumlah HS group pada data adalah 1 : "+str(jumlah_hs_group_1))
print("Jumlah HS religion pada data adalah 1 : "+str(jumlah_hs_religion_1))
print("Jumlah HS physical pada data adalah 1 : "+str(jumlah_hs_physical_1))
print("Jumlah HS gender pada data adalah 1 : "+str(jumlah_hs_gender_1))
print("Jumlah HS others pada data adalah 1 : "+str(jumlah_hs_others_1))
print("-"*50)
print("Jumlah HS individual pada data adalah 0 : "+str(jumlah_hs_individual_0))
print("Jumlah HS group pada data adalah 0 : "+str(jumlah_hs_group_0))
print("Jumlah HS religion pada data adalah 0 : "+str(jumlah_hs_religion_0))
print("Jumlah HS physical pada data adalah 0 : "+str(jumlah_hs_physical_0))
print("Jumlah HS gender pada data adalah 0 : "+str(jumlah_hs_gender_0))
print("Jumlah HS others pada data adalah 0 : "+str(jumlah_hs_others_0))

Jumlah HS individual pada data adalah 1 : 272
Jumlah HS group pada data adalah 1 : 282
Jumlah HS religion pada data adalah 1 : 161
Jumlah HS physical pada data adalah 1 : 134
Jumlah HS gender pada data adalah 1 : 137
Jumlah HS others pada data adalah 1 : 130
--------------------------------------------------
Jumlah HS individual pada data adalah 0 : 364
Jumlah HS group pada data adalah 0 : 354
Jumlah HS religion pada data adalah 0 : 475
Jumlah HS physical pada data adalah 0 : 502
Jumlah HS gender pada data adalah 0 : 499
Jumlah HS others pada data adalah 0 : 506
